In [10]:
#Importando Bibliotecas que estou utilizando para a análise
import pandas as pd
from ydata_profiling import ProfileReport
import os

#Caminho da Base de dados
caminho_dados = r"C:\Users\gabri\OneDrive\Belgeler\GitHub\Python\AnalisesRedebras\Dados\RelPedidoCli.xls"
df_comercial = pd.read_excel(caminho_dados)

# Sobrescreve o dataframe removendo a última linha (Linha total que o próprio ERP ja envia junto)
df_comercial = df_comercial.iloc[:-1]

#------------------------------------------------------------------------------#
#Modificando Número para Texto (Pedidos e N.F -> Precisam ser identificados como texto para a análise)
colunas_texto = ['PEDIDO', 'NF']
for col in colunas_texto:
    df_comercial[col] = (
        df_comercial[col].astype(str)
    )

#------------------------------------------------------------------------------#
# Forçamos as colunas a serem strings para limpeza manual 
# Formatando os Valores de Preços e Custos
colunas_preco = ['TOTAL CUSTO', 'TOTAL ITENS', 'TOTAL ITENS C/ IPI', 'TOTAL PEDIDO', 'QTDE VENDA']
for col in colunas_preco:
    df_comercial[col] = (
        df_comercial[col].astype(str)
    )

for col in colunas_preco:
    # 1. Troca a vírgula pelo ponto decimal (1200,50 -> 1200.50)
    df_comercial[col] = df_comercial[col].str.replace(',', '.', regex=False)
    # 2. Converte para número real (float)
    df_comercial[col] = pd.to_numeric(df_comercial[col], errors='coerce')
    # 3. Arredonda para 2 casas decimais
    df_comercial[colunas_preco] = df_comercial[colunas_preco].round(2)

#------------------------------------------------------------------------------#
#Formatando Colunas de Datas
# Modificando as colunas de Data para o tipo Data
colunas_data = ['DATA', 'NF. EMISSAO']

for col in colunas_data:
    df_comercial[col] = (
    pd.to_datetime(df_comercial[col], dayfirst=True, errors='coerce')
    )

#------------------------------------------------------------------------------#
#Limpeza de Colunas não necessárias para o projeto (Deletando Colunas)
colunas_desnecessarias = ['EMP', 'REPRESENTANTE', 'TIPO', 'INTEGRAÇÃO', 'OUTRAS DESP', 'VLR. FRETE', 'DT. RETIRADA', 'FRETE TYPE']

df_comercial =(
    df_comercial.drop(colunas_desnecessarias,axis=1)
    )

#------------------------------------------------------------------------------#
#Ajustando a coluna de %MKP 
# 1. Garantimos que é string, removemos '%' e espaços
df_comercial['MKP'] = df_comercial['MKP'].astype(str).str.replace('%', '').str.strip()

# 2. Trocamos a vírgula pelo ponto (padrão brasileiro -> americano)
df_comercial['MKP'] = df_comercial['MKP'].str.replace(',', '.')

# 3. Convertemos para número
df_comercial['MKP'] = pd.to_numeric(df_comercial['MKP'], errors='coerce')

# 4. Arredondamos para evitar dízimas (como vimos nas outras colunas)
df_comercial['MKP'] = df_comercial['MKP'].round(2)

#------------------------------------------------------------------------------#
# Criação de colunas
# Lucro R$ (Para saber o lucro de cada venda em dinheiro)
df_comercial['LUCRO R$'] = (
    df_comercial['TOTAL PEDIDO'] - df_comercial['TOTAL CUSTO']
)
df_comercial['LUCRO R$'] = df_comercial['LUCRO R$'].round(2)

In [12]:
df_comercial['TOTAL VENDAS'] = df_comercial['TOTAL PEDIDO'].sum()

In [13]:
df_comercial['TOTAL VENDAS']

0      2453754.54
1      2453754.54
2      2453754.54
3      2453754.54
4      2453754.54
          ...    
650    2453754.54
651    2453754.54
652    2453754.54
653    2453754.54
654    2453754.54
Name: TOTAL VENDAS, Length: 655, dtype: float64